# 1. Scrape all tweets every 3 months!
# 2. Store in csvs locally
# 3. push csvs automatically into repo here!

In [1]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import tweepy
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import streamlit as st
st.set_page_config(layout="wide")


# get all csv file names - already scraped users
extension = 'csv'
all_twitter_user_scraped_csvs = glob.glob(
    'twitterdata\\*.{}'.format(extension))  # CHANGE FOR SHARE STREAMLIT to /
# filter the price csv
all_twitter_user_scraped_csvs = [
    k for k in all_twitter_user_scraped_csvs if 'BITFINEX' not in k]
# filter the price csv
all_twitter_user_scraped_csvs = [
    k for k in all_twitter_user_scraped_csvs if 'relevant_words' not in k]

display_name_all_twitter_user_scraped_csvs = [
    i.split(' ', 1)[0].split('twitterdata\\')[1] for i in all_twitter_user_scraped_csvs]


# map name back to filename:
user_selection_list_containing_twitter_user = [
    k for k in all_twitter_user_scraped_csvs if "100trillionUSD " in k]
user_selection_list_containing_twitter_user



['twitterdata\\100trillionUSD 1303661563291807745.csv',
 'twitterdata\\100trillionUSD 1328299487396327427.csv']

In [2]:
tweet_data = pd.read_csv("{}".format('twitterdata\\cryptocobain 1424552429123670018.csv'))
tweet_data

,id,created_at,text
0,1456958476261986310,2021-11-06 12:15:44+00:00,@Darrenlautf @lawmaster Have fun standing stil...
1,1456943947973120001,2021-11-06 11:18:00+00:00,@PostFlashloan Yeah this guy is a total dick c...
2,1456943369549139981,2021-11-06 11:15:42+00:00,@KGeorgee He told me about it in a tutoring se...
3,1456942533079150599,2021-11-06 11:12:23+00:00,@Nakameowdough There was a group of like 7 fri...
4,1456942212680458242,2021-11-06 11:11:06+00:00,@CatchMeStacking Private island off the east c...
...,...,...,...
3237,1424560557676449794,2021-08-09 02:37:58+00:00,@PepeXBT in the metaverse you're still fat
3238,1424559332889333760,2021-08-09 02:33:06+00:00,"Sushi team unrivaled, got an answer on twitter..."
3239,1424559107663671298,2021-08-09 02:32:12+00:00,@josephdelong @0xMaki THANK YOU ur the best ht...
3240,1424555382161256451,2021-08-09 02:17:24+00:00,@jebus911 Pretentious art kid meets crypto virgin


In [3]:
tweet_data["text"][0] 

'@Darrenlautf @lawmaster Have fun standing still https://t.co/y5m5fsFgIZ'

In [4]:
tweet_data["text"][0][40::].replace(' ', '\n', 1)


'g\nstill https://t.co/y5m5fsFgIZ'

In [5]:
tweet_data["text"][0] 

'@Darrenlautf @lawmaster Have fun standing still https://t.co/y5m5fsFgIZ'

In [7]:
list_of_dfs = []
for user_file in user_selection_list_containing_twitter_user:
    tweet_data = pd.read_csv("{}".format(user_file))
    list_of_dfs.append(tweet_data)

tweet_data = pd.concat(list_of_dfs)
tweet_data

,id,created_at,text
0,1456985396815044615,2021-11-06 14:02:42+00:00,RT @WClementeIII: This week I sat down with bo...
1,1456900938317418497,2021-11-06 08:27:06+00:00,@Mynima4 🌹
2,1456900725548666882,2021-11-06 08:26:15+00:00,@fuserleer 🌹
3,1456898176691097602,2021-11-06 08:16:07+00:00,Movie tip for the weekend: V for Vendetta! It ...
4,1456763288721649667,2021-11-05 23:20:08+00:00,"@geoff_l @Rager I wish that were true, however..."
...,...,...,...
3234,1328640205025382405,2020-11-17 10:04:44+00:00,@blockfolio @BitcoinTina I do the same. Some p...
3235,1328633680856829954,2020-11-17 09:38:48+00:00,@BitcoinTina @blockfolio Correct
3236,1328446169631105025,2020-11-16 21:13:42+00:00,Countdown to 17K🚀 https://t.co/5lXdQaEBCy
3237,1328299873762996224,2020-11-16 11:32:23+00:00,@VisionBitcoin21 IMO we are still in extremist...


In [8]:
for user in user_selection_list_containing_twitter_user:
    # if the file was created longer than 24 hours ago
    if os.path.getmtime(user) + 60*60*24 > time.time():
        print(os.path.getmtime(user))
    # else:
    #     print('Username has been already scraped today.')

1643550825.4855642


In [9]:
user_selection_list_containing_twitter_user = user_selection_list_containing_twitter_user[0]
user_selection_list_containing_twitter_user

'twitterdata\\100trillionUSD 1303661563291807745.csv'

# test tweet on charts

In [15]:
# not working in streamlit, think because takes to long, to many datapoints!

import plotly.express as px
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])

In [28]:
# Calling DataFrame constructor on list
df = pd.DataFrame([0, 1, 2, 3, 4])
df["y"] = [0, 1, 4, 9, 16]
df["img"] = ["C:\\Users\\markus.gebele\\Downloads\\sonst\\twitter lean.png", 
"C:\\Users\\markus.gebele\\Downloads\\sonst\\twitter lean.png", 
"C:\\Users\\markus.gebele\\Downloads\\sonst\\twitter lean.png", 
"C:\\Users\\markus.gebele\\Downloads\\sonst\\twitter lean.png", 
"C:\\Users\\markus.gebele\\Downloads\\sonst\\twitter lean.png"]
df

,0,y,img
0,0,0,C:\Users\markus.gebele\Downloads\sonst\twitter...
1,1,1,C:\Users\markus.gebele\Downloads\sonst\twitter...
2,2,4,C:\Users\markus.gebele\Downloads\sonst\twitter...
3,3,9,C:\Users\markus.gebele\Downloads\sonst\twitter...
4,4,16,C:\Users\markus.gebele\Downloads\sonst\twitter...


In [32]:
df[0]

0    0
1    1
2    2
3    3
4    4
Name: 0, dtype: int64

In [33]:
from PIL import Image

for x, y, png in zip(df[0], df["y"], df["img"]):
    fig.add_layout_image(
        x=x,
        y=y,
        source=Image.open(png),
        xref="x",
        yref="y",
        sizex=1,
        sizey=1,
        xanchor="center",
        yanchor="middle",
    )

In [34]:
fig